In [ ]:
## (Labeling - I) ##

# The labeling in this part was done manually based on the "../../../output/master_code_prep_output/mmr_selected/Q152_mmr_selected.csv" dataset
# A new match column was created and binary hand labeling was performed based on the combined scores (question ID and likert scale)
# Later on the these initial labels would be enriched after the first training run
# The resulting labeled dataset is saved as "Q152_mmr_selected_labeled.csv" in the current directory
# The dataset was the processed to exclude the accompanying metadata and only include the corresponding hashes, and saved as "Q152_mmr_selected_labeled_combined.csv"

%run clean_labeled_data.py

Loading ../../../data/labeled_data/Q152_mmr_selected_labeled.csv ...
Loading ../../../output/master_code_prep_output/top_scored_sentences.csv ...
Creating combined sentence column...
Saving to ../../../data/labeled_data/Q152_mmr_selected_labeled_combined.csv ...
Done.


In [2]:
## (Training) ##

# Note: This code was used twice:
# (1) The first run was made right after the hand labeling was done.
# (2) The second training used a combined labeled dataset of hand labeled data and the hand-picked accurate predictions from the first run by relabeling. 

# Load and filter labeled data, focusing on relevant combined_labels and positive matches.
# Perform stratified train-validation split by label with a fixed holdout fraction.
# Augment training data by adding synonym-replaced sentences to improve robustness.
# Encode sentences with SBERT embeddings and train a CatBoost multi-class classifier.
# Evaluate the model on validation data and save the trained model and label mappings.

%run train_catboost_labeled.py

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\secki\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\secki\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Loading data...
Loading SBERT model...
Encoding all sentences for embedding hash computation...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Splitting data stratified by embedding_hash...
Training samples before augmentation: 69
Validation samples before filtering: 46
Performing synonym augmentation on training set...
Training samples after augmentation: 276
Encoding training sentences after augmentation...


Batches:   0%|          | 0/9 [00:00<?, ?it/s]

Encoding validation sentences...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Encoding original training sentences for permutation tests...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Filtering validation samples too close to training samples...
Validation samples after cosine similarity filtering: 44
Training CatBoost classifier...
0:	learn: 1.3244423	test: 1.3445342	best: 1.3445342 (0)	total: 289ms	remaining: 9m 37s
100:	learn: 0.1658997	test: 0.4816445	best: 0.4816445 (100)	total: 12.9s	remaining: 4m 1s
200:	learn: 0.0857712	test: 0.3934912	best: 0.3934912 (200)	total: 25.6s	remaining: 3m 49s
300:	learn: 0.0577868	test: 0.3616538	best: 0.3616538 (300)	total: 38.7s	remaining: 3m 38s
400:	learn: 0.0440831	test: 0.3333730	best: 0.3333730 (400)	total: 52s	remaining: 3m 27s
500:	learn: 0.0348087	test: 0.3181451	best: 0.3181451 (500)	total: 1m 5s	remaining: 3m 15s
600:	learn: 0.0280449	test: 0.3044470	best: 0.3044470 (600)	total: 1m 18s	remaining: 3m 2s
bestTest = 0.2955469708
bestIteration = 687
Shrink model to first 688 iterations.
Training Multinomial LogLoss at Best Iteration (687): 0.0238
Validation Multinomial LogLoss at Best Iteration (687): 0.2955
Saved model a

In [1]:
# (Prediction Fine-Tuning)
# Optional keyword extraction fine-tuning code using KeyBERT
%run tune_keywords_keybert.py

Top keyphrases for Q152_1:
 - economic expansion fundamental pathway national resilience prosperity (score: 0.753)
 - high level economic growth view sustained economic (score: 0.670)
 - global economy economic vitality engine (score: 0.659)

Top keyphrases for Q152_2:
 - prioritize ensuring country strong defense forces affirm (score: 0.800)
 - capable national defense supports regional stability enables (score: 0.740)
 - prepared defense posture ensure sovereignty peace deterrence (score: 0.737)

Top keyphrases for Q152_3:
 - local councils international forums reforming global governance (score: 0.718)
 - communities legitimacy governance depends meaningfully people (score: 0.675)
 - veto powers overdue empowering communities create solutions (score: 0.671)

Top keyphrases for Q152_4:
 - planet children years come place beauty sustainability (score: 0.709)
 - ecosystems environmental integrity woven public policy rural (score: 0.654)
 - biodiversity combat environmental degradation 

In [ ]:
## (Prediction - I) ##

# This prediction only uses the top_scored_sentences.csv dataset, which is the output of the data_prep_scoring.py script.
# The prediction dataset contains IQR-range candidate sentences in the previous measurements
# As stated, the accurate outputs will be used to enrich the training data for the second training run.

# Load unseen sentences excluding those already labeled in training by embedding hash.
# Use a pre-trained CatBoost model and SBERT embeddings to predict combined_labels on unseen data.
# Filter predicted sentences by confidence threshold (perc_above_chance >= 2.00).
# Merge additional metadata from the full scored sentences dataset based on predicted labels.
# Display a preview and save filtered, labeled predictions to CSV for further use.

%run predict_catboost_top_score.py

Selected 4460 unseen sentences for prediction.
Encoding unseen sentences...


Encoding batches: 100%|██████████| 140/140 [00:04<00:00, 33.71it/s]


Extracting keyphrases per label for semantic filtering...
Applying semantic similarity filtering per predicted label...
Applying joint scoring and filtering...
Kept 842 predictions after joint scoring filtering.
Saved the results to ../../../output/question_pipeline_output/q152_q153_predictions/q152_q153_predictions_top_score_filtered.csv


In [ ]:
## (Labeling - II) ##

# This function appends accurately predicted sentences from the predictions file
# to the labeled dataset based on a provided list of selected sentence hashes (64).
# It filters these hashes to those present in the predictions, then matches them
# with the top scored sentences and marks them as matched.
# Finally, it concatenates these new matched sentences with the existing labeled data and saves it.

# After this code is run, the model was trained the second time with the appended labeled dataset.

#%run label_append_after_pred.py

Total hashes in predictions: 316
Valid selected hashes found in predictions: 0
No valid hashes found in predictions to process.


In [ ]:
## (Training - II) ##

# After the Labeling - II code is run, the model was trained the second time with the appended labeled dataset.

In [5]:
## (Prediction - II) ##

# This code loads UNGA speech sentences and WVC metadata, excluding training data to prevent leakage.
# It encodes filtered sentences with a SentenceTransformer model, then predicts classes and confidences using a CatBoost classifier.
# Predictions with confidence above chance threshold are kept, and relevant columns are selected.
# It merges the predictions with metadata from the WVC dataset based on predicted labels.
# Finally, it previews and saves the combined prediction results to a CSV file for further use.

%run predict_catboost_unga_wvs7.py

Loading combined labeled CSV...
Extracting top keyphrases per label...
Loading SBERT model...
Loading unseen data...
Filtered unseen sentences count: 25588
Loading CatBoost model...
Encoding unseen sentences...


Encoding batches: 100%|██████████| 800/800 [00:32<00:00, 24.27it/s]


Predicting classes and probabilities...
Applying semantic similarity filtering per predicted label...
Applying joint scoring and filtering...
Kept 2317 predictions after joint scoring filtering.
Saved the results to ../../../output/question_pipeline_output/q152_q153_predictions/q152_q153_predictions_filtered.csv


In [6]:
# Predict UNSG speeches using the trained model
%run predict_unsg_address.py

Loading combined labeled CSV...
Extracting top keyphrases per label...
Loading SBERT model...
Loading unseen data...
Loading CatBoost model...
Encoding unseen sentences...


Encoding batches: 100%|██████████| 21/21 [00:00<00:00, 25.52it/s]


Predicting classes and probabilities...
Applying semantic similarity filtering per predicted label...
Applying joint scoring and filtering...
Kept 106 predictions after joint scoring filtering.
Saved the results to ../../../output/question_pipeline_output/q152_q153_predictions/q152_q153_predictions_unsg.csv


In [24]:
## (Post-Processing) ##

# This code analyzes predicted labels per country-year from the predictions dataset,
# identifies the most and second most frequent labels for each country-year (Q152 and Q153, respectively),
# filters to keep only country-years present in the WVS dataset,
# then lists country-years missing a second most frequent label,
# and finally displays and saves the summarized results.

%run get_q152_q153_frequencies.py


Country-year pairs without a second most frequent label:
AND - 2018
CAN - 2020
ECU - 2018
ETH - 2020
JPN - 2019
KOR - 2018
NGA - 2018
NLD - 2022


,B_COUNTRY_ALPHA,A_YEAR,most_frequent_label,most_frequent_count,second_most_frequent_label,second_most_frequent_count,has_second_label
0,AND,2018,Q152_3,4,None,0,False
1,ARG,2017,Q152_1,3,Q152_4,2,True
2,AUS,2018,Q152_2,12,Q152_1,3,True
3,BGD,2018,Q152_2,2,Q152_4,2,True
4,BOL,2017,Q152_2,2,Q152_3,1,True
5,BRA,2018,Q152_2,3,Q152_4,2,True
6,CAN,2020,Q152_3,2,None,0,False
7,CHL,2018,Q152_4,6,Q152_3,4,True
8,CHN,2018,Q152_3,8,Q152_1,6,True
9,COL,2018,Q152_1,1,Q152_2,1,True


Saved the results to ../../../output/question_pipeline_output/q152_q153_output/q152_q153_country_year_top2.csv


In [26]:
# Get the most (Q152) and second most frequent (Q153) labels for each year for the UNSG speeches
%run get_q152_q153_frequencies_unsg.py

Saved to ../../../output/question_pipeline_output/q152_q153_output/q152_year_top2_labels_unsg.csv


,doc_id,A_YEAR,most_frequent_label,most_frequent_count,second_most_frequent_label,second_most_frequent_count,has_second_label
0,AND_72_2017,2017,Q152_3,5,Q152_2,3,True
1,AND_73_2018,2018,Q152_3,4,None,0,False
2,AND_74_2019,2019,Q152_3,7,None,0,False
3,AND_75_2020,2020,Q152_3,2,Q152_2,1,True
4,AND_76_2021,2021,Q152_4,5,Q152_1,1,True
...,...,...,...,...,...,...,...
346,ZWE_73_2018,2018,Q152_1,3,Q152_2,3,True
347,ZWE_74_2019,2019,Q152_1,4,Q152_2,2,True
348,ZWE_75_2020,2020,Q152_3,2,Q152_1,1,True
349,ZWE_76_2021,2021,Q152_1,2,Q152_2,2,True


In [27]:
## (Visualization -I) ##

# This code filters predicted Q152-related labels from the predictions dataset,
# calculates sentence counts and proportions of given response per country,
# prepares the data for ordered plotting,
# then creates and displays a scatter plot showing the proportion of Q152_1 sentences relative to total Q152 sentences by country,
# with point sizes representing sentence counts, x-axis showing the countries and y-axis the proportion of given response to total count responses.

%run visualize_preds_props.py

In [28]:
## (Visualization -II) ##

# This code loads World Values Survey (WVS) data and filters it by countries found in scored sentences.
# It processes Q152 survey responses to calculate per-country counts and proportions of respondents answering '1'.
# It prepares the country data in a categorical order for consistent plotting.
# Then it creates a scatter plot showing the proportion of Q152=1 responses per country, sized by total responses.
# The plot visually compares response distributions across countries with hover details and a clean layout.
# The x and y-axis labels are the same as the prediction visualization.

%run visualize_response_props.py

In [29]:
## (Visualization -III) ##

# This script compares proportions of a specific Q152 survey response between scored sentences and WVS data at the country-year level.
# It aggregates sentence counts and computes proportions per country-year in the scored predictions.
# It filters the WVS data to matching country-year pairs and computes weighted response proportions using survey weights.
# The code then merges both datasets and visualizes their proportions with connecting lines to compare distributions.
# The plot uses country-year labels on the x-axis and proportion values on the y-axis, with vertical lines visually highlighting differences between WVS survey and scored sentence proportions.
# Finally, it calculates and prints the Pearson correlation between the weighted WVS proportions and scored sentence proportions.

%run visualize_prop_diffs.py

Pearson correlation between WVS weighted and scored sentence proportions: 0.0554
